In [1]:
#Loading in our data
!wget https://jmcauley.ucsd.edu/data/amazon_v2/categoryFilesSmall/Gift_Cards_5.json.gz --no-check-certificate
!wget https://jmcauley.ucsd.edu/data/amazon_v2/categoryFilesSmall/Gift_Cards.csv --no-check-certificate
!gzip -d Gift_Cards_5.json.gz
!head Gift_Cards_5.json
!head Gift_Cards.csv
!wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/metaFiles2/meta_Gift_Cards.json.gz --no-check-certificate
!gzip -d meta_Gift_Cards.json.gz
!head meta_Gift_Cards.json

--2023-11-07 10:41:30--  https://jmcauley.ucsd.edu/data/amazon_v2/categoryFilesSmall/Gift_Cards_5.json.gz
Resolving jmcauley.ucsd.edu (jmcauley.ucsd.edu)... 137.110.160.73
Connecting to jmcauley.ucsd.edu (jmcauley.ucsd.edu)|137.110.160.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 176952 (173K) [application/x-gzip]
Saving to: ‘Gift_Cards_5.json.gz’

Gift_Cards_5.json.g 100%[===================>] 172.80K  --.-KB/s    in 0.02s   

2023-11-07 10:41:30 (7.94 MB/s) - ‘Gift_Cards_5.json.gz’ saved [176952/176952]

--2023-11-07 10:41:30--  https://jmcauley.ucsd.edu/data/amazon_v2/categoryFilesSmall/Gift_Cards.csv
Resolving jmcauley.ucsd.edu (jmcauley.ucsd.edu)... 137.110.160.73
Connecting to jmcauley.ucsd.edu (jmcauley.ucsd.edu)|137.110.160.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6143525 (5.9M) [text/csv]
Saving to: ‘Gift_Cards.csv.1’

Gift_Cards.csv.1    100%[===================>]   5.86M  15.0MB/s    in 0.4s    

2023-11-07 

In [1]:
#Installing packages that do not come pre installed
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.13.1+cu116.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.13.1+cu116.html
!pip install torch-geometric
!pip install umap-learn

Looking in links: https://data.pyg.org/whl/torch-1.13.1+cu116.html
  Using cached torch_scatter-2.1.2.tar.gz (108 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp39-cp39-macosx_11_0_arm64.whl size=268030 sha256=e6f251fa9f1c62a6980349854399a7c2fbc75a25352965f8ba1b45597358068e
  Stored in directory: /Users/aidenchang/Library/Caches/pip/wheels/b8/9c/c0/d261e1e64cc49e6b14dd7319d8b61eb14168529d810dbba00e
Successfully built torch-scatter
Looking in links: https://data.pyg.org/whl/torch-1.13.1+cu116.html
  Using cached torch_sparse-0.6.18.tar.gz (209 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for torch-sparse: filename=torch_sparse-0.6.18-cp39-cp39-macosx_11_0_arm64.whl size=463278 sha256=648402f2557253b592a7e08fbc561f3cd1bbb375b68363cef3ab2f02085bcc8c
  Stored in directory: /Users/aidenchang/Library/Caches/pip/wheels/c0/15/18/881c60adefeb60882987c23aa5ecced0ab7257c3162acab518
Successfully built torch-sparse
  Us

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.6/153.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 16.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 41.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 14.3 MB/s eta 0:00:00a 0:00:01


In [1]:
# General libraries
import json
from pathlib import Path as Data_Path
import os
from os.path import isfile, join
import pickle
import random

import numpy as np
import networkx as nx
import pandas as pd
from sklearn.metrics import f1_score, roc_auc_score
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm.notebook import tqdm

In [2]:
from typing import Optional, Union

import torch
from torch import Tensor
import torch.nn as nn
from torch.nn import Embedding, ModuleList, Linear
import torch.nn.functional as F

import torch_geometric
import torch_geometric.nn as pyg_nn
from torch_geometric.data import Data
from torch_geometric.transforms import RandomLinkSplit
from torch.nn.modules.loss import _Loss

from torch_geometric.nn.conv import LGConv, GATConv, SAGEConv
from torch_geometric.typing import Adj, OptTensor, SparseTensor


print(f"Torch version: {torch.__version__}; Torch-cuda version: {torch.version.cuda}; Torch Geometric version: {torch_geometric.__version__}.")

/Users/aidenchang/miniforge3/envs/ml_final/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch version: 2.1.0; Torch-cuda version: None; Torch Geometric version: 2.4.0.


In [3]:
# set the seed for reproducibility
seed = 224
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [4]:
##Defining classes

class Product:
  def __init__(self, title, asin, description, also_bought, categories):
    self.title = title #name, string
    self.asin = asin #string
    self.description = description #string
    # self.features = features # String[]
    self.also_bought = also_bought #asin[]
    self.categories = set(categories) #categories[]


  def __str__(self):
    return f"Product {self.title}"

class Category:
  def __init__(self, name):
    self.name = name
    self.products = []

  def add_to_category(self, product):
    self.products.append(product)

  def __str__(self):
    return f"Category {self.name} has {len(self.products)} in it."

class User:
  def __init__(self, reviewerID):
    self.reviewerID = reviewerID
    self.products_reviewed = []
    #maybe we can add the categories that they reviewed?

  def add_to_reviewed(self, product):
    self.products_reviewed.append(product)

  def __str__(self):
    return f"User {self.reviewerID} has reviewed {len(self.products_reviewed)} products"

In [5]:
import pandas as pd
import json

# reviews = pd.read_csv("Gift_Cards.csv")
review_data = []
review_name = 'Gift_Cards_5.json'
with open(review_name, 'r') as file:
    for line in file:
      review_data.append(json.loads(line))
file_metadata = []
metadata_link = "meta_Gift_Cards.json"
with open(metadata_link, 'r') as file:
    for line in file:
      file_metadata.append(json.loads(line))

In [35]:
review_data[1]

{'overall': 4.0,
 'verified': True,
 'reviewTime': '06 14, 2018',
 'reviewerID': 'A2MN5JQMIY0FQ2',
 'asin': 'B004LLIKVU',
 'style': {'Gift Amount:': ' 50'},
 'reviewerName': 'Amazon Customer',
 'reviewText': 'Gift card for my daughter',
 'summary': 'Ease of use',
 'unixReviewTime': 1528934400}

In [36]:
products = {}
all_categories = {}
users = {}

def process_review_data(review):
  try:
    userID = review['reviewerID']
  except:
    print(review)
  if userID not in users:
    new_user = User(userID)
    users[userID] = new_user
  rating = int(review['overall'])
  users[userID].add_to_reviewed((review['asin'], rating))

def process_metadata(metadata):
  asin = metadata['asin']
  # features = metadata['features']
  also_bought = metadata['also_buy']
  description = metadata['description']
  categories = metadata['category']
  title = metadata['title']
  if asin not in products:
    new_product = Product(title, asin, description, also_bought, categories)
    products[asin] = new_product
  for category in categories:
    if category not in all_categories:
      all_categories[category] = Category(category)
    all_categories[category].add_to_category(asin)


def load_data():
  for line in review_data:
    process_review_data(line)
  for line in file_metadata:
    process_metadata(line)


In [37]:
load_data()

In [38]:
import torch
from torch_geometric.data import Data

# Create instances of your classes
# This is just a placeholder. Replace this with your actual data.
# products = [...]
# categories = [...]
# users = [...]

# Create mappings from unique IDs to consecutive integers
unique_user_id = [user for user in users]
unique_user_id = pd.DataFrame(data={
   'userId': unique_user_id,
   'mappedID': pd.RangeIndex(len(unique_user_id)),
})

unique_product_id = [product for product in products]
unique_product_id = pd.DataFrame(data={
   'productId': unique_product_id,
   'mappedID': pd.RangeIndex(len(unique_product_id)),
})

unique_category_id = [category for category in all_categories]
unique_category_id = pd.DataFrame(data={
   'categoryId': unique_category_id,
   'mappedID': pd.RangeIndex(len(unique_category_id)),
})
# print(unique_user_id)
# Construct the edge indices
# User to Product (Review)
user_product_edges = []
user_product_ratings = []
for user_name in users:
   user_object = users[user_name]
   for product_id, rating in users[user_name].products_reviewed:
      product = products[product_id]
      user_product_edges.append([unique_user_id.loc[unique_user_id['userId'] == user_object.reviewerID, 'mappedID'].values[0],
                              unique_product_id.loc[unique_product_id['productId'] == product.asin, 'mappedID'].values[0]])
      user_product_ratings.append(rating)
user_product_edges = torch.t(torch.tensor(user_product_edges, dtype=torch.long))
user_product_ratings = torch.tensor(user_product_ratings, dtype=torch.float) 
# user_product_ratings = torch.reshape(user_product_ratings, (1, len(user_product_ratings)))

print(user_product_edges.shape)

# Product to Product (Also Bought)
product_product_edges = []
for product_id in products:
   product = products[product_id]
   for also_bought_product in products[product_id].also_bought:
      #  print(also_bought_product)
 
       if (unique_product_id['productId'] == also_bought_product).any():  
         product_product_edges.append([unique_product_id.loc[unique_product_id['productId'] == product.asin, 'mappedID'].values[0],
                                   unique_product_id.loc[unique_product_id['productId'] == also_bought_product, 'mappedID'].values[0]])
# print(len(product_product_edges))
product_product_edges = torch.t(torch.tensor(product_product_edges, dtype=torch.long))

# Product to Category
product_category_edges = []
for product_id in products:
   product = products[product_id]
   for category in products[product_id].categories:
       product_category_edges.append([unique_product_id.loc[unique_product_id['productId'] == product.asin, 'mappedID'].values[0],
                                    unique_category_id.loc[unique_category_id['categoryId'] == category, 'mappedID'].values[0]])

product_category_edges = torch.t(torch.tensor(product_category_edges, dtype=torch.long))

torch.Size([2, 2972])


In [9]:
print(len(product_product_edges), len(product_category_edges), len(product_category_edges))



2 2 2


In [10]:
from sentence_transformers import SentenceTransformer

class TextEncoder:
    """
    A class for encoding text using a SentenceTransformer model.
    """
    def __init__(self, model='all-MiniLM-L6-v2', device=None):
        """
        :param model: Name of the SentenceTransformer model to use.
        :param device: Device to use for model inference. Default is None.
        """
        self.device = device
        self.model = SentenceTransformer(model, device=self.device)

    @torch.no_grad()
    def __call__(self, values: list):
        """
        Encode a list of text values into embeddings.

        :param values: List of text values to encode.
        :return: Encoded embeddings as a PyTorch tensor.
        """
        x = self.model.encode(values,
                              show_progress_bar=True,
                              convert_to_tensor=True,
                              device=self.device)
        return x.cpu()


# Check if CUDA is available, and set the device accordingly
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
# Create an instance of the TextEncoder class with the determined device
encoder = TextEncoder(device=device)

In [11]:
ex = "I am aiden"
e = encoder(ex)
print(e.shape)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 14.14it/s]

torch.Size([384])


In [12]:
products[list(products.keys())[0]].description

['Gift card for the purchase of goods or services at Serendipity 3 in New York City only. Not valid for online purchases.',
 'Statements regarding dietary supplements have not been evaluated by the FDA and are not intended to diagnose, treat, cure, or prevent any disease or health condition.']

In [13]:
# Create product features
product_features = torch.zeros((len(unique_product_id), 384))
for product_id in products:
   product = products[product_id]
   mapped_id = unique_product_id.loc[unique_product_id['productId'] == product.asin, 'mappedID'].values[0]
   d = ""
   for descrip in product.description:
      d = d + descrip
   product.description = d
   product_features[mapped_id] = encoder(product.description)

Batches: 100%|██████████| 1/1 [00:00<00:00, 81.50it/s]


In [14]:
product_features[-1].size()

torch.Size([384])

In [39]:
from torch_geometric.data import HeteroData
from sklearn.feature_extraction.text import TfidfVectorizer

# Create node feature tensors
# product_features = torch.tensor([products[product].description for product in products], dtype=torch.float)
# user_features = torch.tensor([user.rating for user in users], dtype=torch.float)

# Create an instance of the HeteroData class
data = HeteroData()

# Add node features to the HeteroData object
data['Product'].x = product_features
data['User'].x = torch.ones(len(users), 1)
data['Category'].x = torch.ones(len(all_categories), 1)
# data['User'].x = user_features

# Add edge indices to the HeteroData object
data['User', 'REVIEWS', 'Product'].edge_index = user_product_edges
data['User', 'REVIEWS', 'Product'].edge_attr = user_product_ratings
data['Product', 'ALSO_BOUGHT', 'Product'].edge_index = product_product_edges
data['Product', 'HAS_CATEGORY', 'Category'].edge_index = product_category_edges

In [19]:
# data['Product'].x = product_features
# data['User'].x = user_features

In [60]:
data
# n, e = data.metadata()
# print(n)
# print(e)
# data['Category'].num_nodes



HeteroData(
  Product={ x=[1547, 384] },
  User={ x=[458, 1] },
  Category={ x=[14, 1] },
  (User, REVIEWS, Product)={ edge_index=[2, 2972] },
  (Product, ALSO_BOUGHT, Product)={ edge_index=[2, 5993] },
  (Product, HAS_CATEGORY, Category)={ edge_index=[2, 3622] }
)

In [40]:
import torch_geometric.transforms as T


# t_data = T.ToUndirected()(data)
# t_data = T.ToUndirected()(data)
# t_data = T.ToUndirected()(data)

transform = RandomLinkSplit(
    is_undirected=False, 
    add_negative_train_samples=False, 
    disjoint_train_ratio=0.3,
    neg_sampling_ratio=0,
    num_val=0.1, num_test=0.1,
    edge_types=["REVIEWS","ALSO_BOUGHT", 'HAS_CATEGORY']
)

#we have to do the split for each edge type and combine them back together

# transform = RandomLinkSplit(is_undirected=True, edge_types=["REVIEWS","ALSO_BOUGHT", 'HAS_CATEGORY'])
train_split, val_split, test_split = transform(data)
# train_split, val_split, test_split = transform(data)

In [41]:
torch.Tensor.type(train_split['User', 'REVIEWS', 'Product'].edge_index[0]) #64bit int
# type(test_split['User', 'REVIEWS', 'Product'].edge_index[0][0])

'torch.LongTensor'

In [28]:
from torch_geometric.utils import negative_sampling
n = negative_sampling(train_split['User', 'REVIEWS', 'Product'].edge_index, num_neg_samples=1)
print(train_split['User', 'REVIEWS', 'Product'].edge_index, n)

tensor([[  77,   73,  336,  ...,   62,  184,  139],
        [ 446,  194, 1040,  ...,  703,  274,  359]]) tensor([[663],
        [292]])


In [26]:
from torch_geometric.nn.conv import lg_conv
model = LGConv(normalize = True)

In [27]:
model.forward()

LGConv()

In [33]:
train_split['REVIEWS']['edge_index'].shape

torch.Size([2, 2378])

In [ ]:
def sample_negative_edges_hetero(data, num_users, num_products, device=None):
    # For simplicity, let's focus on the 'REVIEWS' relationship as an example.
    # You would need to adapt this for other types of relationships in your graph.
    positive_users, positive_products = data['REVIEWS']['edge_index']

    # Create a mask tensor with the shape (num_users, num_products)
    mask = torch.zeros(num_users, num_products, device=device, dtype=torch.bool)
    mask[positive_users, positive_products] = True  # No need to offset indices for hetero graph

    # Flatten the mask tensor and get the indices of the negative edges
    flat_mask = mask.flatten()
    negative_indices = torch.where(~flat_mask)[0]

    # Sample negative edges from the negative_indices tensor
    sampled_negative_indices = negative_indices[
        torch.randint(0, negative_indices.size(0), size=(positive_users.size(0),), device=device)
    ]

    # Convert the indices back to users and products tensors
    users = torch.div(sampled_negative_indices, num_products, rounding_mode='floor')
    products = sampled_negative_indices % num_products

    neg_edge_index = torch.stack((users, products), dim=0)
    neg_edge_label = torch.zeros(neg_edge_index.shape[1], device=device)

    return neg_edge_index, neg_edge_label